In [1]:
import math
import datetime
import os, sys
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from IPython.display import Audio
import csv

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torchsummary import summary

import cv2




# local files
sys.path.insert(0, '../')
import pyClient
import utils
import model
from model import Transition

In [2]:
# Training parameters
BATCH_SIZE = 128 #original 128
GAMMA = 0.999
EPS_START = 0.95
EPS_END = 0.05
EPS_DECAY_steps = 4000 
EPS_DECAY = (EPS_START - EPS_END)/EPS_DECAY_steps
REPLAY_START_SIZE =  128 #TODO PUT BACK TO 1500 #steps taken
TARGET_UPDATE = 10 #episodes
DEVICE = 'cpu'

# Environment parameters
IMSIZE = 128
STACK_SIZE = 1
N_ACTIONS = 3
IP  = "127.0.0.1" # Ip address that the TCP/IP interface listens to
PORT = 13000       # Port number that the TCP/IP interface listens to


environment = pyClient.Environment(ip = IP, port = PORT, size = IMSIZE) 
agent = model.DoubleDQNAgent(imsize=IMSIZE,
                 in_channels=STACK_SIZE,
                 n_actions=N_ACTIONS,
                 memory_capacity=12000,
                 eps_start=EPS_START,
                 eps_end=EPS_END,
                 eps_delta=EPS_DECAY,
                 gamma_discount = GAMMA,
                 batch_size = BATCH_SIZE,
                 device=DEVICE)



In [3]:
# Utilities
def process_state(state_raw):
    """ @TODO """
    frame = environment.state2usableArray(state_raw)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = frame.astype('float32')
    return torch.Tensor(frame / 255.).view(1,1,environment.size, environment.size)

In [ ]:
MAX_EPISODES = 800
MAX_STEPS  = 130
TRAINING_CONDITION = 0
LOGFILE = 'Experiments/Out/test-01.csv'

RESET_UPON_END_SIGNAL = {0:False,  # Nothing happened
                         1:True,   # Box collision
                         2:True,   # Wall collision
                         3:False}  # Reached step target



with open(LOGFILE, 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['episode','step_count', 'train_loss', 'reward'])

lr_dqn = 0.01
optimizer = optim.Adam(agent.policy_net.parameters(), lr = lr_dqn)
total_reward = 0
total_loss = 0
for episode in range(MAX_EPISODES):
    with open(LOGFILE, 'a') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        writer.writerow([episode,agent.step_count,total_reward,total_loss]) 
    
    
    if agent.step_count > MAX_STEPS:
        break
    
    if episode % TARGET_UPDATE == 0:  #episodes
        print('target net updated')
        agent.update_target_net
    
    print('episode {}'.format(episode))
    _, _, state_raw = environment.reset(TRAINING_CONDITION)
    state = process_state(state_raw)

    for t in count(): 
        action = agent.select_action(state)
        end, reward, next_state_raw = environment.step(action.item())
        next_state = process_state(next_state_raw) if not RESET_UPON_END_SIGNAL[end] else None
        if reward > 100:
            reward = -(reward -100)
        reward = torch.tensor([reward], device=DEVICE,dtype=torch.float)
        total_reward += reward.item();
        action = action.unsqueeze(0)
        agent.memory.push(state, action, next_state, reward)

        if RESET_UPON_END_SIGNAL[end]:
            break
            
        if len(agent.memory) > REPLAY_START_SIZE:
            
            state_action_values, expected_state_action_values = agent.forward()
            
            """ @TODO: Optimize model"""
            # Compute Huber loss
            loss = F.smooth_l1_loss(state_action_values, expected_state_action_values)
            total_loss += loss.item()

            # Optimize the model
            optimizer.zero_grad()
            loss.backward()

            nn.utils.clip_grad_norm_(agent.policy_net.parameters(), 1)

            optimizer.step()
        else:
            agent.step_count = 0

target net updated
episode 0
